# Bit vector arithmetics (verification)

The cyrite library and kernel is coming from a signal processing background where arithmetic operations are pipelined. This implies extended requirements to signal data types.

The MyHDL originating `intbv` data was regarded as 'state of the art' with respect to simplicity and numeric stability, however there are still issues to be pointed out when conversion to VHDL and Verilog comes into play. This is a short introduction giving you an idea what to watch out for.

### The intbv data type

An intbv is a priori an integer value in a certain range. In hardware, it is a bit vector. Let us just define two signals using `intbv` wires. We can here use different notations:

In [1]:
from cyhdl import *

a = Signal(intbv(min=-12, max = 18), name = 'a')
bu = Signal(intbv()[7:], name = 'bu')
bs = Signal(intbv()[7:].signed(), name = 'bs')
c = Signal(intbv()[8:], name = 'c')
d = Signal(intbv()[12:].signed(), name = 'd')

For `a`, the length is implicit:

In [2]:
len(a), a.is_signed()

(6, True)

We now define a few expressions of additions and use the .evaluate() function to verify the arithmetics.

Here in interactive mode we don't use the `.next` style assignment but the internal representations's '.set()' method:

In [3]:
assign = bs.set(0x40)

Evaluating this assignment actually performs the initialization, but it is expected to fail:

In [4]:
try:
    assign.evaluate()
except ValueError as e:
    print("Error:", e.args)

Error: ('intbv value 64 >= maximum 64',)


We try again and run an addition:

In [5]:
statements = [
    bs.set(0x20),
    c.set(bs + bs)
]

We observe though that `bs` is a signed value whereas the result `c` is unsigned. If is was signed, we would get an overflow.
When evaluating this statement chain, we should see a `0x40` as a result:

In [6]:
for stmt in statements:
    v = stmt.evaluate()

hex(v)

'0x40'

This is just what Python does. When we translate this operation to hardware elements, we may wish to verify it's correctly calculating as well. Because we normally emit to a established HDL understood by hardware synthesis tools, we would like to check this behaviour against its HDL edition.

## HDL translation and simulation

To output statements to a V*HDL in a granular way, the DummyTargetModule is imported. Emission of statements results in HDL output to the standard output.

In [7]:
from myirl.targets.dummy import DummyTargetModule

d = DummyTargetModule(targets.VHDL)
# Try targets.Verilog as alternative

for stmt in statements:
    stmt.emit(d)

bs <= "0100000";
c <= unsigned((bs + signed(resize(bs, 8))));


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <c> <= ADD(bs, bs)
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


### Simulation

For the simulation and verification, we may want to pack the above test sequence into a function. We modify it however to add values that are initialized as unsigned, while being cased to signed values before addition.

The most generic way to write a functional description that is reusable in various ways is to use a RTL function. A RTL function is always encapsulated by a class structure.

Note that we use the '.next' assignment notation here. The reason is that the code below has a dual function, depending on the RTL context:

1. It can run as native Python code with Simulation type signals
2. It can transpile to HDL, hence it is translated to IRL (the intermediate representation)

In [8]:
class RTLMethods:
    @rtl_function
    def arith_test1(rtl, a, b, q, val):
        a.next = val[0]
        b.next = val[1]
        yield rtl.delay(1)
        q.next = a.signed() + b.signed()
        yield rtl.delay(1)

A RTL function receives the current RTL context as first parameter, not a class instance of `RTLMethods`.

When called from a `cyrite_factory.Module` class, `self` is passed as `rtl` context. To allow customization of the delay, it is good practise to use a `yield rtl.delay(1)` call to pass control to the simulator.

We write a cyrite_factory module as follows:

In [9]:
from cyrite.simulation import sim, ghdl, icarus

class MyDesign(cyrite_factory.Module):
    _rtl = RTLMethods
    def delay(self, val):
        return delay(val)
    @cyrite_factory.testbench('ns')
    def tb_arith(self):
        @self.sequence
        def main():
            yield from self._rtl.arith_test1(self.a, self.b, self.q, self.initval)
            print("RESULT", self.q)
            assert self.q == self.expected_result
            print("Simulation done.")
            raise StopSimulation
    
        return instances()
        

Then we call the test bench. We add one bit of head room to the destination signal in order to avoid truncation.

In [10]:
m = MyDesign("design", ghdl.GHDL)
m.a, m.b = ( Signal(intbv()[7:], name = n) for n in "ab")
m.q = Signal(intbv()[8:], name = 'c')

m.initval, m.expected_result = (0x40, 0x40), 0x80
tb = m.tb_arith()
tb.run(200, debug = True)

 Declare obj 'tb_arith' in context '(MyDesign 'design')'(<class '__main__.MyDesign'>) 
 Writing 'tb_arith' to file /tmp/tb_arith.vhdl 
 Creating library file /tmp/module_defs.vhdl 
DEBUG_FILES ['/tmp/tb_arith.vhdl', '/tmp/module_defs.vhdl']


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <c> <= ADD(SGN(<a>), SGN(<b>))
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


==== COSIM stdout ====
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/tb_arith.vhdl
elaborate tb_arith

==== COSIM stdout ====
RESULT 0x80
Simulation done.
simulation stopped @2ns



0

We do this again with different values, to verify signed casting does the right thing.

In [11]:
m.initval, m.expected_result = (0x3f, -0x20 & 0x7f), 0x1f
tb.run(200, debug = True)

 Writing 'tb_arith' to file /tmp/tb_arith.vhdl 
 Creating library file /tmp/module_defs.vhdl 
DEBUG_FILES ['/tmp/tb_arith.vhdl', '/tmp/module_defs.vhdl']


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <c> <= ADD(SGN(<a>), SGN(<b>))
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


==== COSIM stdout ====
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/tb_arith.vhdl
elaborate tb_arith

==== COSIM stdout ====
RESULT 0x1F
Simulation done.
simulation stopped @2ns



0

## Custom arithmetics

We can define our own integer type by derivation from the BuiltinIntType. However, this does not necessarily guarantee for correct HDL inference. separate fixup routines may be needed. We illustrate this with a simple class that only implements the '__add__' operation.

In [12]:
from myirl.kernel.extensions import BuiltinIntType
from myirl.kernel import sig as base

class MyIntAddOnly(BuiltinIntType):
    def __init__(self, val):
        if isinstance(val, int):
            self._val = val
            self._nbits = val.bit_length()
            if val < 0:
                self._signed = True
            else:
                self._signed = False
        elif isinstance(val, str):
            mval = val.replace('_', '')
            self._val = int(mval, 2)
            self._nbits = len(mval)
        else:
            raise TypeError("Unsupported argument", type(val))
    
    def __add__(self, other):
        return self._val + other._val

    def __int__(self):
        return self._val

    def __len__(self):
        return self._nbits

    def size(self, effective = None):
        return self._nbits

    def signal_type(self, tgt, size = None):
        if size is None:
            size = self.size()
        n = size - 1
        if self.is_signed():
            typestr = tgt.type_signed_vector % (n)	
        else:
            typestr = tgt.type_vector % (n)
        return typestr

    def is_signed(self):
        return self._signed

We initialize two signals of this wire type:

In [13]:
s = Signal(MyIntAddOnly(32), name = 's')
t = Signal(MyIntAddOnly(64), name = 't')

We check their bit vector size:

In [14]:
len(s), len(t)

(6, 7)

In [15]:
z = s.signed() + s.signed()
len(z)

7

We then write an explicit addition logic statement and emit it to the stdout VHDL translator:

In [16]:
add_s = t.set(s.signed() + s.signed())

In [17]:
add_s.emit(d)

t <= unsigned((signed(s) + signed(s)));


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <t> <= ADD(SGN(<s>), SGN(<s>))
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


In [18]:
c.set(bs.signed() + bs.signed()).emit(d)

c <= unsigned((signed(bs) + signed(resize(signed(bs), 8))));


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <c> <= ADD(SGN(<bs>), SGN(<bs>))
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


This construct looks somewhat non-obvious to non-VHDL experts with respect to what will happen to the bits in particular.

Now re run the simulation with these rudimentary signal types:

In [19]:
def run_sim_hdl(sim, s, t):
    m = MyDesign("design", sim)
    m.a, m.b = ( Signal(MyIntAddOnly(32), name = 's%d' % i) for i in range(2) )
    m.q = t
    m.initval, m.expected_result = (0x20, 0x20), 0x40
    tb = m.tb_arith()
    tb.run(200, debug = True)

In [20]:
try:
    run_sim_hdl(ghdl.GHDL, s, t)
except Exception as e:
    print(e)

 Declare obj 'tb_arith' in context '(MyDesign 'design')'(<class '__main__.MyDesign'>) 
 Writing 'tb_arith' to file /tmp/tb_arith.vhdl 
 Creating library file /tmp/module_defs.vhdl 
DEBUG_FILES ['/tmp/tb_arith.vhdl', '/tmp/module_defs.vhdl']


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <t> <= ADD(SGN(<s0>), SGN(<s1>))
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


==== COSIM stdout ====
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/tb_arith.vhdl
elaborate tb_arith

==== COSIM stdout ====
/tmp/tb_arith:error: bound check failure at /tmp/tb_arith.vhdl:32
in process .tb_arith(irl_uncached).main
/tmp/tb_arith:error: simulation failed

Simulation run failed




For example: A 6 bit vector with value 0x20 has only its MSB set. When casted to signed, this is interpreted as -32. Adding this vector to itself results in -64 which requires 7 bits when casted back to unsigned: 0x40. If we would use a 6 bit falue for the result, the bound check error will disappear, and the result will be truncated to 0x00 which is expected.

### Verilog version

The same emitted to a Verilog simulation will not cause an error. This is because Verilog handles vector length extension more implicitely.

Conclusion: For HDL portability, integer data types need internal fixups.

In [21]:
run_sim_hdl(icarus.ICARUS, s, t)

 Declare obj 'tb_arith' in context '(MyDesign 'design')'(<class '__main__.MyDesign'>) 
 Writing 'tb_arith' to file /tmp/tb_arith.v 
 Note: Changing library path prefix to /tmp/ 
 Creating library file /tmp/module_defs.v 
DEBUG FILES ['/tmp/tb_arith.v']


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <t> <= ADD(SGN(<s0>), SGN(<s1>))
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


==== COSIM stdout ====
VCD info: dumpfile tb_arith.vcd opened for output.
RESULT 0b1000000 
Simulation done. 
Stop Simulation



## Semi-automated verification

Above, we specified the result we expected. However, we could also let Python evaluate the code and determine the expected result automatically when emitting to HDL.

Internally, this uses a modified sequence generator using an evaluating context as below:

In [22]:
from myirl.library.verification import _EvalContext, Checkpoint
from myirl.simulation import sequential

class my_generator(sequential):

	def __repr__(self):
		return "[SimGeneratorCTX `%s`]" % (self.func.__name__)
	"yield based generator process with context"

	def __call__(self, ctx):
		self.sequence = _EvalContext(ctx)
		self.sequence.inherit(ctx)
		ret = self.func(self.sequence)
		self._collect(ctx, ret)

		return ret


We then rewrite the MyDesign class slightly, by using a `self.check` function. This inserts Checkpoint generators that evaluate the argument via native python and emit a comparison statement to HDL where the signal is checked for its expected value that was determined by evaluation of the statement sequence.

In [23]:
class MyDesignAuto(MyDesign):

    check = Checkpoint
    
    def __init__(self, name, target):
        super().__init__(name, target)
        if self.translate:
            self.sequence = my_generator

    @cyrite_factory.testbench('ns')
    def tb_arith(self):
        @self.sequence
        def main():
            yield from self._rtl.arith_test1(self.a, self.b, self.q, self.initval)
            print("RESULT", self.q)
            # assert self.q == self.expected_result
            self.check(self.q, "Verifying result of addition")
            print("Simulation done.")
            raise StopSimulation

        return instances()

Then we run the design again as above:

In [24]:
m = MyDesignAuto("design", ghdl.GHDL)
m.a, m.b = bs, bs
m.q = c
m.initval, m.expected_result = (0x20, 0x20), 0x40
tb = m.tb_arith()
tb.run(200, debug = True)

 Declare obj 'tb_arith' in context '(MyDesignAuto 'design')'(<class '__main__.MyDesignAuto'>) 
 Writing 'tb_arith' to file /tmp/tb_arith.vhdl 
 Creating library file /tmp/module_defs.vhdl 
DEBUG_FILES ['/tmp/tb_arith.vhdl', '/tmp/module_defs.vhdl']


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:104: UserWarning: Implicit sign conversion <c> <= ADD(SGN(<bs>), SGN(<bs>))
  base.warnings.warn("Implicit sign conversion %s <= %s" % (repr(self), repr(v)))


==== COSIM stdout ====
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/tb_arith.vhdl
elaborate tb_arith

==== COSIM stdout ====
RESULT 0x40
VAL 0x40
Simulation done.
simulation stopped @3ns



0

Inspection of the resulting VHDL test bench displays the Checkpoint generator sequence (encapsulated by `{}` brackets):

In [25]:
!cat /tmp/tb_arith.vhdl

-- File generated from source:
--     /tmp/ipykernel_82334/3187735675.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity tb_arith is
end entity tb_arith;

architecture irl_uncached of tb_arith is
    -- Local type declarations
    -- Signal declarations
    signal bs : signed(6 downto 0);
    signal c : unsigned(7 downto 0);
begin
    
main:
    process
    begin
        bs <= "0100000";
        bs <= "0100000";
        wait for 1 ns;
        c <= unsigned((signed(bs) + signed(resize(signed(bs), 8))));
        wait for 1 ns;
        print("RESULT" & " " & "0x"& hstr(c));
        -- Checkpoint Verifying result of addition {
        wait for 1 ns;
        print("VAL" & " " & "0x"& hstr(c));
        assert (c = x"40")
            report "Verifying result of addition -- expected: 64" severity failure;
  

### Limitations

The auto-verification sequencer is incomplete in this release and will not work inside generated for or while loops, obviously. In more complex cases, it might be better to use co-simulation.

# intbv versus bv

The `bv` type differs from the MyHDL originating intbv:

In [26]:
from myirl.emulation.bv import bv

In [45]:
a0, a1 = [ Signal(bv()[6:]) for _ in range(2) ]
q = Signal(bv()[9:].signed())

In [46]:
m.a, m.b = (a0, a1)
m.q = q

m.initval, m.expected_result = (0x20, 0x20), 0x40
tb.run(200, debug = True)

 Writing 'tb_arith' to file /tmp/tb_arith.vhdl 
 Creating library file /tmp/module_defs.vhdl 
DEBUG_FILES ['/tmp/tb_arith.vhdl', '/tmp/module_defs.vhdl']
==== COSIM stdout ====
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/tb_arith.vhdl
elaborate tb_arith

==== COSIM stdout ====
RESULT 0x000
VAL 0x000
/tmp/tb_arith.vhdl:55:9:@3ns:(assertion failure): Verifying result of addition -- expected: -64
/tmp/tb_arith:error: assertion failed
in process .tb_arith(irl_uncached).main
/tmp/tb_arith:error: simulation failed



RuntimeError: Simulation run failed

In [ ]:
!cat -n /tmp/tb_arith.vhdl